In [45]:
import QuantLib as ql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import math
from tqdm import tqdm
import time

In [25]:
'''
    ################ CVXPY Solution ############
    Phi = cp.Variable((ncolloc, ));
    expr = 0;
    
    dS = shift(delta_S_hat.loc[:, t].values, 1, cval=0); dS = np.diag(dS)
    constraints = [data_mat_t[t,:,:]@Phi <= 1, data_mat_t[t,:,:]@Phi >= -1]
    alpha = 0.1*np.abs(np.mean(dS))
    Pih = shift(Pi_hat.loc[:, t].values, 1, cval=0);
    expr = cp.sum(-dS@data_mat_t[t, :, :]@Phi) + cp.sum(Pih - dS@data_mat_t[t, :, :]@Phi) + alpha*cp.square(cp.norm(data_mat_t[t,:,:]@Phi-prev, 2))
    
    objective = cp.Minimize(expr);
    problem = cp.Problem(objective, constraints);
    opt = problem.solve('ECOS');
    Phi = Phi.value;
    prev =  np.dot(data_mat_t[t,:,:],Phi)
    a.loc[:,t] = prev
    Pi.loc[:,t] = gamma * (Pi.loc[:,t+1] - a.loc[:,t] * delta_S.loc[:,t])
    Pi_hat.loc[:,t] = Pi.loc[:,t] - np.mean(Pi.loc[:,t])
    ###############################################
'''

"\n    ################ CVXPY Solution ############\n    Phi = cp.Variable((ncolloc, ));\n    expr = 0;\n    \n    dS = shift(delta_S_hat.loc[:, t].values, 1, cval=0); dS = np.diag(dS)\n    constraints = [data_mat_t[t,:,:]@Phi <= 1, data_mat_t[t,:,:]@Phi >= -1]\n    alpha = 0.1*np.abs(np.mean(dS))\n    Pih = shift(Pi_hat.loc[:, t].values, 1, cval=0);\n    expr = cp.sum(-dS@data_mat_t[t, :, :]@Phi) + cp.sum(Pih - dS@data_mat_t[t, :, :]@Phi) + alpha*cp.square(cp.norm(data_mat_t[t,:,:]@Phi-prev, 2))\n    \n    objective = cp.Minimize(expr);\n    problem = cp.Problem(objective, constraints);\n    opt = problem.solve('ECOS');\n    Phi = Phi.value;\n    prev =  np.dot(data_mat_t[t,:,:],Phi)\n    a.loc[:,t] = prev\n    Pi.loc[:,t] = gamma * (Pi.loc[:,t+1] - a.loc[:,t] * delta_S.loc[:,t])\n    Pi_hat.loc[:,t] = Pi.loc[:,t] - np.mean(Pi.loc[:,t])\n    ###############################################\n"

In [26]:
'''
## model calibration ##
param_0 = np.array([0.25, 5, 0.2, 0.1]);
tau = np.arange(0, 30 + 1/12, 1/12)

data = data_dict(df, 1)
term = TS('Vasicek', data, param_0, tau)
term.calibration()
p1 = term.p;

data = data_dict(df, 60)
term = TS('Vasicek', data, param_0, tau)
term.calibration()
p2 = term.p;

data = data_dict(df, 120)
term = TS('Vasicek', data, param_0, tau)
term.calibration()
p3 = term.p;

data = data_dict(df, 180)
term = TS('Vasicek', data, param_0, tau)
term.calibration()
p4 = term.p;

plt.plot(range(len(p1)), p1, label="+0yr")
plt.plot(range(len(p2)), p2, label="+5yr")
plt.plot(range(len(p3)), p3, label="+10yr")
plt.plot(range(len(p4)), p4, label="+15yr")
plt.legend()
plt.xlabel('Maturity (mo)')
plt.ylabel('Zero-coupon Discount Yield')
plt.title('Term Structure at Varying Dates')
'''

'\n## model calibration ##\nparam_0 = np.array([0.25, 5, 0.2, 0.1]);\ntau = np.arange(0, 30 + 1/12, 1/12)\n\ndata = data_dict(df, 1)\nterm = TS(\'Vasicek\', data, param_0, tau)\nterm.calibration()\np1 = term.p;\n\ndata = data_dict(df, 60)\nterm = TS(\'Vasicek\', data, param_0, tau)\nterm.calibration()\np2 = term.p;\n\ndata = data_dict(df, 120)\nterm = TS(\'Vasicek\', data, param_0, tau)\nterm.calibration()\np3 = term.p;\n\ndata = data_dict(df, 180)\nterm = TS(\'Vasicek\', data, param_0, tau)\nterm.calibration()\np4 = term.p;\n\nplt.plot(range(len(p1)), p1, label="+0yr")\nplt.plot(range(len(p2)), p2, label="+5yr")\nplt.plot(range(len(p3)), p3, label="+10yr")\nplt.plot(range(len(p4)), p4, label="+15yr")\nplt.legend()\nplt.xlabel(\'Maturity (mo)\')\nplt.ylabel(\'Zero-coupon Discount Yield\')\nplt.title(\'Term Structure at Varying Dates\')\n'

In [27]:
'''
phi = cp.Variable((12,))
expr = cp.square(cp.norm(A_mat@phi - B_vec)) + lmbda*cp.norm(cp.diff(data_mat_t[t,:,:]@phi, axis=0), 1)
objective = cp.Minimize(expr)
constraints = [];
problem = cp.Problem(objective, constraints)
opt = problem.solve();
phi = phi.value
'''


'\nphi = cp.Variable((12,))\nexpr = cp.square(cp.norm(A_mat@phi - B_vec)) + lmbda*cp.norm(cp.diff(data_mat_t[t,:,:]@phi, axis=0), 1)\nobjective = cp.Minimize(expr)\nconstraints = [];\nproblem = cp.Problem(objective, constraints)\nopt = problem.solve();\nphi = phi.value\n'

In [12]:
'''
import QuantLib as ql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


sigma = 0.01
a = 0.1
length = 30 # in years
steps_per_year = 12
timestep = length * steps_per_year
forward_rate = 0.01
day_count = ql.Thirty360(ql.Thirty360.BondBasis)
todays_date = ql.Date(15, 1, 2015)


ql.Settings.instance().evaluationDate = todays_date

spot_curve = ql.FlatForward(todays_date, ql.QuoteHandle(ql.SimpleQuote(forward_rate)), day_count)
spot_curve_handle = ql.YieldTermStructureHandle(spot_curve)

hw_process = ql.HullWhiteProcess(spot_curve_handle, a, sigma)
rng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(timestep, ql.UniformRandomGenerator()))
seq = ql.GaussianPathGenerator(hw_process, length, timestep, rng, False)


def generate_paths(n_scenarios):
    arr = np.zeros((n_scenarios, timestep+1))
    for i in range(n_scenarios):
        sample_path = seq.next()
        path = sample_path.value()
        time = [path.time(j) for j in range(len(path))]
        value = [path[j] for j in range(len(path))]
        arr[i, :] = np.array(value)
    return np.array(time), arr

n_scenarios = 1000
time, paths = generate_paths(n_scenarios)
rates = pd.DataFrame(paths).T

#price a zero coupon bond
n_years = length
num_steps = timestep
dt = 1 / steps_per_year
prices = np.empty_like(rates)


def price(T, t, f0, rt):
    tau = T - t
    B = (1 - np.exp(-a * tau)) / a
    A = np.exp(-f0 * tau + B * f0 - sigma ** 2 / (4 * a ** 3) *
               (np.exp(-a * T) - np.exp(-a * t)) * (np.exp(2 * a * t) - 1))
    return A * np.exp(-rt * B)

prices[0] = price(n_years,dt*0, forward_rate, rates.values[0])

for steps in range(1,num_steps+1):
    prices[steps] = price(n_years, dt * steps, forward_rate, rates.values[steps])


fig = plt.figure()
plt.figure(figsize=(16,10))

plt.subplot(2, 1, 1)
plt.title("All ZCB paths")
plt.plot(prices)

plt.subplot(2, 1, 2)

ave = [np.mean(x) for x in prices]
plt.plot(ave)
plt.title("Average ZCB paths")

plt.show()
'''

'\nimport QuantLib as ql\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport pandas as pd\n\n\nsigma = 0.01\na = 0.1\nlength = 30 # in years\nsteps_per_year = 12\ntimestep = length * steps_per_year\nforward_rate = 0.01\nday_count = ql.Thirty360(ql.Thirty360.BondBasis)\ntodays_date = ql.Date(15, 1, 2015)\n\n\nql.Settings.instance().evaluationDate = todays_date\n\nspot_curve = ql.FlatForward(todays_date, ql.QuoteHandle(ql.SimpleQuote(forward_rate)), day_count)\nspot_curve_handle = ql.YieldTermStructureHandle(spot_curve)\n\nhw_process = ql.HullWhiteProcess(spot_curve_handle, a, sigma)\nrng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(timestep, ql.UniformRandomGenerator()))\nseq = ql.GaussianPathGenerator(hw_process, length, timestep, rng, False)\n\n\ndef generate_paths(n_scenarios):\n    arr = np.zeros((n_scenarios, timestep+1))\n    for i in range(n_scenarios):\n        sample_path = seq.next()\n        path = sample_path.value()\n        time = [path.t

# Montonicity Check of Yield Curve

In [2]:
import math

def calibration_report(swaptions, vols_data):
    
    columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol", "Rel Er\
    ror Price", "Rel Error Vols", "Expiry", "Maturity"]
    
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0
    
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = vols_data[i]['vol']
        black_price = s.blackPrice(market_vol)
        rel_error = model_price/black_price - 1.0
        implied_vol = s.impliedVolatility(model_price,
        1e-1, 50, 0, 1.00)
     
        rel_error2 = implied_vol/market_vol-1.0
        cum_err += rel_error*rel_error
        cum_err2 += rel_error2*rel_error2
        report_data.append((model_price, black_price, implied_vol, market_vol, rel_error, rel_error2, vols_data[i]['expiry'], vols_data[i]['maturity']))
        
    #print("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
    #print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
    
    return pd.DataFrame(report_data,columns= columns, index=['']*len(report_data))

In [145]:
## Term Structure Model ##
from QuantLib import *
import pandas as pd 
class TS:
    def __init__(self, calibration_path, vols_path, payment_periods, tstart, num_paths, notional):
        self.term_structure = self.build_curve(calibration_path, tstart);
        self.calibrated_model, gauss_generator = self.calibrate(vols_path, notional, payment_periods, tstart)
        _, self.short_rate_paths = self.generate_paths(num_paths, gauss_generator)
        #plt.plot(self.short_rate_paths.T)
        
    def generate_paths(self, num_paths, seq):
        timestep = 10800
        arr = np.zeros((num_paths, timestep+1))
        for i in range(num_paths):
            sample_path = seq.next()
            path = sample_path.value()
            time = [path.time(j) for j in range(len(path))]
            value = [path[j] for j in range(len(path))]
            arr[i, :] = np.array(value)
        return np.array(time), arr   
    
    def build_curve(self, calibration_path, tstart):
        df = pd.read_csv(calibration_path)
        data = []
        for i in range(len(df)):
            dt = dict(df.loc[i])
            dt['Date'] = '2017-01-01 00:00:00'
            data.append(dt)

        calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
        zeros = []
        deposits = ['1M', '3M', '6M', '12M']
        swaps = ['2Y', '3Y', '5Y', '7Y', '10Y', '15Y', '20Y', '30Y']
        for row in data[tstart:tstart+1]:
          
            # Build Curve for the date
            curve_date = ql.Date(row['Date'][:10], '%Y-%m-%d')
            ql.Settings.instance().evaluationDate = curve_date
            spot_date = calendar.advance(curve_date, 0, ql.Days) # beginning of eval - 2 days forward
            helpers = ql.RateHelperVector()
            for tenor in deposits:
                index = ql.USDLibor(ql.Period(tenor))
                helpers.append(
                    ql.DepositRateHelper(row[tenor] / 100, index)
                )
            for tenor in swaps:
                swap_index = ql.UsdLiborSwapIsdaFixAm(ql.Period(tenor))
                swapHelper = ql.SwapRateHelper(row[tenor]/100, swap_index)
                helpers.append(
                    ql.SwapRateHelper(row[tenor] / 100, swap_index)
                )
            curve = ql.PiecewiseCubicZero(curve_date, helpers, ql.Actual360())
        curve.enableExtrapolation()
        term_structure = ql.YieldTermStructureHandle(curve)
        
        return term_structure

    def calibrate(self, vols_path, notional, payment_periods, tstart):
      
        timestep = 10800
        model = ql.HullWhite(self.term_structure);
        engine = ql.JamshidianSwaptionEngine(model)

        index = ql.Euribor6M(self.term_structure)
        fixedLegTenor = ql.Period(payment_periods, ql.Months) # originally 1Y
        fixedLegDayCounter = ql.Actual360()
        floatingLegDayCounter = ql.Actual360()

        ## Grab volatilities ##
        df  = pd.read_csv(vols_path)
        date1 = df.loc[tstart] # Index important here
        volcub = np.reshape(date1[1:].values, (7,2))
        tenor_labels = [_ for _ in [5, 10]]
        expiry_labels = [_ for _ in [1/12, 3/12, 6/12, 1, 2, 5, 10]]
        self.swaption_vols = pd.DataFrame(volcub, index=expiry_labels, columns=tenor_labels)
        
        swaptions = []
        ql.Settings.instance().evaluationDate = ql.Date(1, 1, 2017)
        vols_data = []
        
        cnt = 0
        for maturity in self.swaption_vols.index:
            for tenor in self.swaption_vols.columns:
                vol = self.swaption_vols.at[maturity, tenor] / 100
                if(vol != 0):
                        vols_data.append({'vol':vol, 'expiry':maturity, 'maturity':tenor})
                        volatility = ql.QuoteHandle(ql.SimpleQuote(vol))
                        helper = ql.SwaptionHelper(
                            ql.Period(int(maturity*12), ql.Months),
                            ql.Period(int(tenor), ql.Years),
                            volatility,
                            index,
                            fixedLegTenor,
                            fixedLegDayCounter,
                            floatingLegDayCounter,
                            self.term_structure,
                            ql.BlackCalibrationHelper.RelativePriceError,
                            ql.nullDouble(),
                            notional, # nominal
                            ql.ShiftedLognormal, # ShiftedLogn
                            0.00 #shift to make rates non-negative
                        )
                        helper.setPricingEngine(engine)
                        swaptions.append(helper)
                        glob[(maturity, tenor)] = [cnt, vol];
                        cnt += 1
                else:
                    raise ValueError("0 vol. provied!")
    

        optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
        end_criteria = ql.EndCriteria(500000, 1000, 1e-6, 1e-8, 1e-8)
        model.calibrate(swaptions, optimization_method, end_criteria)
        
        self.params = model.params()
        
        df = calibration_report(swaptions, vols_data)
        #display(df)
        
        self.sigma = self.params[1]
        self.a = self.params[0]
        #print(params)
        length = 30 # in years
        day_count = ql.Thirty360(ql.Thirty360.BondBasis)
        todays_date =  Date(1, 1, 2017)

        ql.Settings.instance().evaluationDate = todays_date

        hw_process = ql.HullWhiteProcess(self.term_structure, self.a, self.sigma)
        rng = ql.GaussianLowDiscrepancySequenceGenerator(ql.UniformLowDiscrepancySequenceGenerator(timestep))
        seq = ql.GaussianSobolPathGenerator(hw_process, length, timestep, rng, False)

        #rng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(timestep, ql.UniformRandomGenerator()))
        #seq = ql.GaussianPathGenerator(hw_process, length, timestep, rng, False)
        
        self.swaptions = swaptions;
            
        return model, seq
        
    def ZCBCurve(self, t, k=None):
        """ P(t, T) for all t, T """
        #print("Implied discount = ", self.calibrated_model.discount(t))
        if(k is not None):
            path = self.short_rate_paths[k, :];
            PTt = [self.calibrated_model.discountBond(t, i+t, path[int(t*360)]) for i in np.arange(0, 30, 1/360)]
            return PTt;
        PTts = []
        for k in range(len(self.short_rate_paths)):
            path = self.short_rate_paths[k,:]
            rates = []
            for i in np.arange(0, 30, 1/360):
                rate = self.calibrated_model.discountBond(t, i+t, path[int(t*360)])
                rates.append(rate)
            PTts.append(rates);
        PTts = np.vstack(PTts);
        num_traj, traj_length = PTts.shape
        PTtsAvg = np.average(PTts, axis=0)
        return PTtsAvg

In [ ]:
#calibration_path, vols_path, payment_periods, tstart, num_paths, notional
tsh = TS('bloomberg_calibration.csv', 'vols_abridged.csv', 6, 0, 1000, 1e8)
plt.figure(2)
for i in np.arange(0, 10800, 3600):
    print("done 1")
    plt.plot(tsh.ZCBCurve(i/360), '--', label=str(i/360))
    pass
plt.legend()


# Zero Coupon Bond Pricing

In [149]:

N_MC = 8192
tsh = TS('bloomberg_calibration.csv', 'vols_liquid.csv', 3, 0, N_MC, 1e8)
print(tsh.params)

## Check recovery of discount bond prices ##
from QuantLib import *


day_count = ql.Thirty360(ql.Thirty360.BondBasis)
calendar = calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

spot_curve_handle = tsh.term_structure #YieldTermStructureHandle(spot_curve)

# Construct bond schedule
issue_date = Date(1, 1, 2017)
maturity_date = Date(2, 1, 2027)
tenor = Period(Annual)

business_convention = Unadjusted
date_generation = DateGeneration.Backward
month_end = False

schedule = Schedule(issue_date, maturity_date, tenor, calendar, business_convention, business_convention, date_generation, month_end)

# Create FixedRateBond Object

coupon_rate = 0.00
coupons = [coupon_rate]
settlement_days = 0
face_value = 1

fixed_rate_bond = FixedRateBond(settlement_days,
                                face_value,
                                schedule,
                                coupons,
                                day_count)

# Set Valuation engine
bond_engine = DiscountingBondEngine(spot_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

# Calculate present value
forward_times = [1/12, 2/12, 3/12, 6/12, 12/12, 2, 5, 10]
for ft in forward_times:
    tindex = int(ft * 360)
    value = tsh.ZCBCurve(0)[tindex] #fixed_rate_bond.NPV()
    print("Quantlib Analytical price = ", value)

    MCvalue = 0
    for i in range(N_MC):
        MCvalue += 1/np.exp(sum(tsh.short_rate_paths[i, 0:tindex])*1/360)
    MCvalue /= N_MC
    print("Monte carlo ZCB price  = ", MCvalue)
    print("Rel. error (bps) = ", (MCvalue/value - 1)*10000)


[ 0.0440413; 0.0110728 ]
Quantlib Analytical price =  0.9956713561141977
Monte carlo ZCB price  =  0.9956713778791162
Rel. error (bps) =  0.0002185954062206008
Quantlib Analytical price =  0.9912106897818841
Monte carlo ZCB price  =  0.991217283715574
Rel. error (bps) =  0.06652403730100787
Quantlib Analytical price =  0.9863911343836547
Monte carlo ZCB price  =  0.9864000294750018
Rel. error (bps) =  0.09017813560108223
Quantlib Analytical price =  0.972080220543652
Monte carlo ZCB price  =  0.9720902027854242
Rel. error (bps) =  0.10268948551050983
Quantlib Analytical price =  0.9431239573862075
Monte carlo ZCB price  =  0.9431334321753465
Rel. error (bps) =  0.1004617586564116
Quantlib Analytical price =  0.9271502322699198
Monte carlo ZCB price  =  0.9271071156284004
Rel. error (bps) =  -0.46504482249631884
Quantlib Analytical price =  0.8535856853234498
Monte carlo ZCB price  =  0.8535053999107559
Rel. error (bps) =  -0.9405665309802913
Quantlib Analytical price =  0.7475445959513

# Swap Pricing 

In [147]:
class swap:
    def __init__(self, forward_time, maturity, fixed_sched, floating_sched, fixed_rate, tau_n, notional, tstart, N_MC=100):

        self.forward_time = forward_time;
        self.maturity = maturity;
        self.fixed_sched = fixed_sched;
        self.floating_sched = floating_sched;
        self.fixed_rate = fixed_rate;
        self.tau_n = tau_n;
        self.notional = notional

        calibration_path = 'bloomberg_calibration.csv'
        vols_path = 'vols_liquid.csv'
        num_paths = N_MC
        #tstart = 0
        self.tsh = TS(calibration_path, vols_path, int(self.tau_n * 360), tstart, num_paths, notional)
        
        self.term_structure, fixed_sched = self.advance_dynamics(0)
        if(fixed_rate is None):
            A = sum([self.term_structure[int(fixed_sched[i])] for i in range(len(fixed_sched)-1)])
            B = sum([self.term_structure[int(fixed_sched[i+1])] for i in range(len(fixed_sched)-1)])
            self.fixed_rate = (A-B)/(self.tau_n*B)
        else:
            self.fixed_rate = fixed_rate
        #print("Value at zero = ", self.value_at_t(0))
        #plt.figure(1)
        #plt.plot(tau, self.term_structure)
        #print("params = ", term.par)

    def advance_dynamics(self, t, k=None):
        p = self.tsh.ZCBCurve(t, k);
        return p, self.fixed_sched + (self.forward_time - t)*360

    def value_at_t(self, t, k=None):

        # calibrate to current time t #
        term_structure, fixed_sched = self.advance_dynamics(t, k)
        annuity = sum([term_structure[int(i)] for i in fixed_sched])*self.tau_n 
        v_swap = 0;
        for date in range(0, len(fixed_sched)-1):
            v_swap += (term_structure[int(fixed_sched[date])] - term_structure[int(fixed_sched[date+1])] - self.tau_n*self.fixed_rate*term_structure[int(fixed_sched[date+1])])
        return v_swap*self.notional, ((v_swap)/annuity + self.fixed_rate) # (swap value, swap rate)
    
    def payoff(self):
        return self.value_at_t(self.maturity + self.forward_time)

In [148]:
from QuantLib import *
forward_times = [1/12, 3/12, 6/12, 1, 2, 5, 10]
maturities = [5, 10]
for ft in forward_times:
    for m in maturities:
        forward_time = ft#1/12
        maturity = m#5
        pp = 3; # months
        payment_periods = pp * 30
        fixed_sched = np.arange(0, maturity*360 + payment_periods, payment_periods)
        floating_sched = fixed_sched
        tau_n = payment_periods/360
        tstart = 0
        N_MC = 8192
        notional = 1
        fixed_rate = 0.05
        s1 = swap(forward_time, maturity, fixed_sched, floating_sched, fixed_rate, tau_n, notional, tstart, N_MC)
        mc = 0
        v_swap = 0
        dF = np.exp(-np.sum(np.array(s1.tsh.short_rate_paths[:, 0:int(forward_time*360)]), axis=1)*1/360)
        for i in range(N_MC):
            mc += s1.value_at_t(forward_time, i)[0]*dF[i]

        ## Check recovery of swap prices ##
        # Build Swap
        today_date = Date(1, 1, 2017)
        settlement_date = today_date + ql.Period(int(forward_time*12), ql.Months)
        maturity_date = settlement_date + ql.Period(int(maturity*12), ql.Months) 
        ql.Settings.instance().evaluationDate = today_date

        discount_curve = s1.tsh.term_structure
        fwd_curve = s1.tsh.term_structure
        calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
        fixed_schedule = ql.Schedule(settlement_date, maturity_date, ql.Period(pp, ql.Months), calendar, ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Backward, False)
        float_schedule = ql.Schedule(settlement_date, maturity_date, ql.Period(pp, ql.Months), calendar, ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Backward, False)
        libor3M_index = ql.USDLibor(ql.Period('3M'), fwd_curve)


        fixed_leg_daycount = ql.Thirty360(ql.Thirty360.BondBasis)
        float_spread = 0
        float_leg_daycount = ql.Thirty360(ql.Thirty360.BondBasis)

        ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, fixed_rate, fixed_leg_daycount, float_schedule, libor3M_index, float_spread, float_leg_daycount)

        swap_engine = ql.DiscountingSwapEngine(discount_curve)
        ir_swap.setPricingEngine(swap_engine)

        myprice = mc/N_MC
        rerr =(myprice/ir_swap.NPV() - 1)*100

        print('Total NPV: {}'.format(ir_swap.NPV()))
        #print('Premium: {}'.format(-1*ir_swap.NPV()/notional*100))
        print('QLBS Swap Price: {}'.format(myprice))
        print('Rel. error: {} %'.format(rerr))

Total NPV: -0.08316973795739324
QLBS Swap Price: -0.08300632479518495
Rel. error: -0.19648151626016075 %
Total NPV: -0.1757301714173531
QLBS Swap Price: -0.17600412873576834
Rel. error: 0.15589657496242193 %
Total NPV: -0.08775585136402853
QLBS Swap Price: -0.08746765844677121
Rel. error: -0.32840307828800874 %
Total NPV: -0.17964513403207458
QLBS Swap Price: -0.17978878213834967
Rel. error: 0.07996214706791349 %
Total NPV: -0.09502510789140914
QLBS Swap Price: -0.0947270887870812
Rel. error: -0.31362143220978744 %
Total NPV: -0.18585524204274242
QLBS Swap Price: -0.1860175558726568
Rel. error: 0.0873334688494154 %
Total NPV: -0.11046123749045215
QLBS Swap Price: -0.11009588341828877
Rel. error: -0.33075319493407473 %
Total NPV: -0.19906178938011013
QLBS Swap Price: -0.1993207717870716
Rel. error: 0.1301015166034425 %
Total NPV: -0.09901949812338359
QLBS Swap Price: -0.09918574927150418
Rel. error: 0.16789738513260932 %
Total NPV: -0.18362469720906036
QLBS Swap Price: -0.18421216897262

# Swaption Pricing

In [150]:
def run_sim(forward_time, maturity, fixed_rate, N_MC, notional, tau_n, payment_periods, tstart):
    fixed_sched = np.arange(0, maturity*360 + 2*payment_periods, payment_periods)
    print("Naive payment schedule = ", fixed_sched)
    floating_sched = fixed_sched;
    tau_n = tau_n
    s1 = swap(forward_time, maturity, fixed_sched, floating_sched, fixed_rate, tau_n, notional, tstart, N_MC)
    trajectories = []
    rates = []
    for n in tqdm(range(N_MC)):
        time.sleep(0.1)
        s1vals = [];
        s1rates = []
        x = []
        # n = k #
        val, rate = s1.value_at_t(forward_time, n)
        trajectories.append(val)
        rates.append(rate)
    return trajectories, rates, s1

    # set up sim params
forward_times = [1/12, 3/12, 6/12, 1, 2, 5, 10]
maturities = [5, 10]
for ft in forward_times:
    for mat in maturities:
        glob = dict()
        forward_time = ft
        maturity = mat
        pp = 3; # months
        payment_periods = pp * 30
        tau_n = payment_periods/360
        tstart = 0
        N_MC = 8192
        notional = 1
        fixed_rate = None

        # run main sim 
        trajectories, rates, s1 = run_sim(forward_time, maturity, fixed_rate, N_MC, notional, tau_n, payment_periods, tstart)
        print(glob)

        # get index #
        meta = glob[(forward_time, maturity)]; ind = meta[0]; vol = meta[1]

        ## Validate monte carlo estimate with analytical via discounting
        muv  = np.maximum(np.array(trajectories), 0)

        dF = np.exp(-np.sum(np.array(s1.tsh.short_rate_paths[:, 0:int(forward_time*360)]), axis=1)*1/360)
        print("Avg. Numeraire = ", np.average(dF))
        mcest = np.average(np.multiply(muv, dF))
        trt = s1.tsh.swaptions[ind].modelValue()

        #plt.figure(2)
        #plt.plot(np.multiply(muv, dF), '.')
        #plt.plot([mcest]*N_MC) 

        ## check against synthetic swaption (must run previous swap valuation cell) ##
        swaption = ql.Swaption(ir_swap, ql.EuropeanExercise(settlement_date), ql.Settlement.Physical)
        engine = ql.BlackSwaptionEngine(s1.tsh.term_structure, ql.QuoteHandle(ql.SimpleQuote(vol)))
        swaption.setPricingEngine(engine)
        qlprice = swaption.NPV()

        print("Synethtic instr. price = ", qlprice)
        print("Model Estimate = ", trt)
        print("Monte Carlo Estimate = ", mcest)
        print("Rel. error (to Model est) = %.2f" % (100*(mcest/trt - 1)) + " %")

Naive payment schedule =  [   0   90  180  270  360  450  540  630  720  810  900  990 1080 1170
 1260 1350 1440 1530 1620 1710 1800 1890]


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [16:20<00:00,  8.36it/s]


{(0.08333333333333333, 5): [0, 0.34], (0.08333333333333333, 10): [1, 0.316], (0.25, 5): [2, 0.33590000000000003], (0.25, 10): [3, 0.3208], (0.5, 5): [4, 0.3427], (0.5, 10): [5, 0.3274], (1.0, 5): [6, 0.3593], (1.0, 10): [7, 0.33770000000000006], (2.0, 5): [8, 0.3807], (2.0, 10): [9, 0.35109999999999997], (5.0, 5): [10, 0.3469], (5.0, 10): [11, 0.3211], (10.0, 5): [12, 0.2967], (10.0, 10): [13, 0.2984]}
Avg. Numeraire =  0.9956713765547784
Synethtic instr. price =  0.03622421618920598
Model Estimate =  0.005466586328076494
Monte Carlo Estimate =  0.005096154738067677
Rel. error (to Model est) = -6.78 %
Naive payment schedule =  [   0   90  180  270  360  450  540  630  720  810  900  990 1080 1170
 1260 1350 1440 1530 1620 1710 1800 1890 1980 2070 2160 2250 2340 2430
 2520 2610 2700 2790 2880 2970 3060 3150 3240 3330 3420 3510 3600 3690]


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [16:22<00:00,  8.34it/s]


{(0.08333333333333333, 5): [0, 0.34], (0.08333333333333333, 10): [1, 0.316], (0.25, 5): [2, 0.33590000000000003], (0.25, 10): [3, 0.3208], (0.5, 5): [4, 0.3427], (0.5, 10): [5, 0.3274], (1.0, 5): [6, 0.3593], (1.0, 10): [7, 0.33770000000000006], (2.0, 5): [8, 0.3807], (2.0, 10): [9, 0.35109999999999997], (5.0, 5): [10, 0.3469], (5.0, 10): [11, 0.3211], (10.0, 5): [12, 0.2967], (10.0, 10): [13, 0.2984]}
Avg. Numeraire =  0.9956713765547784
Synethtic instr. price =  0.031329170112413186
Model Estimate =  0.009401956764614723
Monte Carlo Estimate =  0.008445280706276748
Rel. error (to Model est) = -10.18 %
Naive payment schedule =  [   0   90  180  270  360  450  540  630  720  810  900  990 1080 1170
 1260 1350 1440 1530 1620 1710 1800 1890]


...

{(0.08333333333333333, 5): [0, 0.34], (0.08333333333333333, 10): [1, 0.316], (0.25, 5): [2, 0.33590000000000003], (0.25, 10): [3, 0.3208], (0.5, 5): [4, 0.3427], (0.5, 10): [5, 0.3274], (1.0, 5): [6, 0.3593], (1.0, 10): [7, 0.33770000000000006], (2.0, 5): [8, 0.3807], (2.0, 10): [9, 0.35109999999999997], (5.0, 5): [10, 0.3469], (5.0, 10): [11, 0.3211], (10.0, 5): [12, 0.2967], (10.0, 10): [13, 0.2984]}
Avg. Numeraire =  0.8535879209536374
Synethtic instr. price =  0.03236392736431778
Model Estimate =  0.05539171687567273
Monte Carlo Estimate =  0.05166433416841322
Rel. error (to Model est) = -6.73 %
Naive payment schedule =  [   0   90  180  270  360  450  540  630  720  810  900  990 1080 1170
 1260 1350 1440 1530 1620 1710 1800 1890]


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [16:27<00:00,  8.29it/s]


{(0.08333333333333333, 5): [0, 0.34], (0.08333333333333333, 10): [1, 0.316], (0.25, 5): [2, 0.33590000000000003], (0.25, 10): [3, 0.3208], (0.5, 5): [4, 0.3427], (0.5, 10): [5, 0.3274], (1.0, 5): [6, 0.3593], (1.0, 10): [7, 0.33770000000000006], (2.0, 5): [8, 0.3807], (2.0, 10): [9, 0.35109999999999997], (5.0, 5): [10, 0.3469], (5.0, 10): [11, 0.3211], (10.0, 5): [12, 0.2967], (10.0, 10): [13, 0.2984]}
Avg. Numeraire =  0.7472680350896136
Synethtic instr. price =  0.02745084432269267
Model Estimate =  0.03584081495446025
Monte Carlo Estimate =  0.03435881177901709
Rel. error (to Model est) = -4.13 %
Naive payment schedule =  [   0   90  180  270  360  450  540  630  720  810  900  990 1080 1170
 1260 1350 1440 1530 1620 1710 1800 1890 1980 2070 2160 2250 2340 2430
 2520 2610 2700 2790 2880 2970 3060 3150 3240 3330 3420 3510 3600 3690]


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [16:27<00:00,  8.30it/s]


{(0.08333333333333333, 5): [0, 0.34], (0.08333333333333333, 10): [1, 0.316], (0.25, 5): [2, 0.33590000000000003], (0.25, 10): [3, 0.3208], (0.5, 5): [4, 0.3427], (0.5, 10): [5, 0.3274], (1.0, 5): [6, 0.3593], (1.0, 10): [7, 0.33770000000000006], (2.0, 5): [8, 0.3807], (2.0, 10): [9, 0.35109999999999997], (5.0, 5): [10, 0.3469], (5.0, 10): [11, 0.3211], (10.0, 5): [12, 0.2967], (10.0, 10): [13, 0.2984]}
Avg. Numeraire =  0.7474128130441109
Synethtic instr. price =  0.02778967497310643
Model Estimate =  0.061898901435214065
Monte Carlo Estimate =  0.05718788751315858
Rel. error (to Model est) = -7.61 %


In [6]:
# Python3 implementation of QuickSort


# Function to find the partition position
def partition(array, low, high):

    # Choose the rightmost element as pivot
    pivot = array[high]

    # Pointer for greater element
    i = low - 1
    print(i, pivot)
    # Traverse through all elements
    # compare each element with pivot
    for j in range(low, high):
        if array[j] <= pivot:

            # If element smaller than pivot is found
            # swap it with the greater element pointed by i
            i = i + 1

            # Swapping element at i with element at j
            (array[i], array[j]) = (array[j], array[i])

    # Swap the pivot element with
    # the greater element specified by i
    (array[i + 1], array[high]) = (array[high], array[i + 1])
    
    # Return the position from where partition is done
    print(i+1)
    return i + 1


# Function to perform quicksort
def quicksort(array, low, high):
    if low < high:

        # Find pivot element such that
        # element smaller than pivot are on the left
        # element greater than pivot are on the right
        pi = partition(array, low, high)
        print(low, high, pi)
        print("")

        # Recursive call on the left of pivot
        quicksort(array, low, pi - 1)

        # Recursive call on the right of pivot
        quicksort(array, pi + 1, high)



array = [10, 7, 8, 9, 1, 5]
N = len(array)

# Function call
quicksort(array, 0, N - 1)
print('Sorted array:')
for x in array:
    print(x, end=" ")

# This code is contributed by Adnan Aliakbar

-1 5
1
0 5 1

1 7
2
2 5 2

2 8
3
3 5 3

3 9
4
4 5 4

Sorted array:
1 5 7 8 9 10 